<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>

# Plot variables relevant to vegetation growth and biomass from ELM output.

#### ----------------------------------------------------------------------------------------------------------------------

<br>

To run this script and generate the resulting ELM plots we need specific Python libraries.  This step loads those required libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray
import os
import ipywidgets as widgets
import glob, numpy

Next, we locate the output data and identify which sites there is output for. This code creates a dropdown menu to pick the model run you want to plot.

In [ ]:
output_rootdir=os.path.expanduser('~')+'/output/cime_run_dirs/'
cases=sorted(glob.glob("%s*20TR*" % output_rootdir))
cases=[os.path.basename(x) for x in cases]
cases_dropdown = widgets.Dropdown(options=cases,
                                description='Choose Case Name:',
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

display(cases_dropdown)

Here, we are using the combined output file over all years of the run. We read in the list of variables, units, and longer descriptions from the dataset. We use the time variable from the dataset to determine the start and end years.

In [ ]:
output_file=output_rootdir + cases_dropdown.value + '/run/ELM_output.nc'

# Read in the list of variable, units, and longer descriptions from the dataset
with xarray.open_dataset(output_file) as data:
    # Determine start and end year of the dataset
    data_startyear=data.time[0].item().year
    data_endyear=data.time[-1].item().year

startyear_picker = widgets.BoundedIntText(value=data_startyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose start year',style={'description_width':'auto'})
endyear_picker = widgets.BoundedIntText(value=data_endyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose end year',style={'description_width':'auto'})

smoothing_picker = widgets.IntText(value=0,description='Choose smoothing on fluxes (months)',style={'description_width':'auto'})

display(startyear_picker)
display(endyear_picker)
display(smoothing_picker)

Using the fields above, you can pick the start and end year of the time series to visualize.

Next, we read in the dataset and select the time portion based on start and end year picked above.

In [ ]:
year_start=startyear_picker.value
year_end=endyear_picker.value
smoothing=smoothing_picker.value

elm_output=xarray.open_dataset(output_file).squeeze().sel(time=slice(str(year_start),str(year_end)))

Now we set up a figure and plot the relevant variables in different panels.

In [ ]:
# Plot the carbon and nitrogen budgets
# Set up a figure with three axes
fig,a=plt.subplots(nrows=2,ncols=2,clear=True,num='Carbon budgets',figsize=(15,13))

ax=a[0,0]
elm_output['TOTVEGC'].plot(ax=ax,linestyle='-',color='black',label='Total vegetation C')
elm_output['LEAFC'].plot(ax=ax,linestyle='-',color='green',label='Leaf C')
elm_output['FROOTC'].plot(ax=ax,linestyle='-',color='orange',label='Fine root C')
(elm_output['DEADSTEMC']+elm_output['LIVESTEMC']+elm_output['DEADCROOTC']+elm_output['LIVECROOTC']).plot(ax=ax,linestyle='-',color='brown',label='Woody C')
ax.legend()
ax.set(title='Veg carbon pools',xlabel='Year',ylabel='Carbon stock (g C m$^{-2}$)')


# Vegetation N
ax=a[0,1]
elm_output['TOTVEGN'].plot(ax=ax,linestyle='-',color='black',label='Total vegetation C')
elm_output['LEAFN'].plot(ax=ax,linestyle='-',color='green',label='Leaf C')
elm_output['FROOTN'].plot(ax=ax,linestyle='-',color='orange',label='Fine root C')
# This is showing a negative N content for live coarse roots, which seems like a model issue we should check on...
(elm_output['DEADSTEMN']+elm_output['LIVESTEMN']+elm_output['DEADCROOTN']+elm_output['LIVECROOTN']).plot(ax=ax,linestyle='-',color='brown',label='Woody C')
ax.set(title='Veg nitrogen pools',xlabel='Year',ylabel='Nitrogen stock (g N m$^{-2}$)')


# C fluxes
ax=a[1,0]
# Do smoothing if wanted
if smoothing>0:
    NEE=elm_output['NEE'].resample(time=str(smoothing)+'M').mean()
    NPP=elm_output['NPP'].resample(time=str(smoothing)+'M').mean()
    HR=elm_output['HR'].resample(time=str(smoothing)+'M').mean()
else:
    NEE=elm_output['NEE']
    NPP=elm_output['NPP']
    HR=elm_output['HR']

(NEE*24*3600).plot(ax=ax,linestyle='-',color='black',label='Net ecosystem exchange')
(NPP*24*3600).plot(ax=ax,linestyle='-',color='green',label='Net primary production')
(HR*24*3600).plot(ax=ax,linestyle='-',color='brown',label='Ecosystem respiration')
ax.legend()
ax.set(title='C fluxes',xlabel='Year',ylabel='C flux (g C m$^{-2}$ day$^{-1}$)')

# N fluxes
ax=a[1,1]
# Do smoothing if wanted
if smoothing>0:
    NET_NMIN=elm_output['NET_NMIN'].resample(time=str(smoothing)+'M').mean()
    GROSS_NMIN=elm_output['GROSS_NMIN'].resample(time=str(smoothing)+'M').mean()
    SMINN_TO_PLANT=elm_output['SMINN_TO_PLANT'].resample(time=str(smoothing)+'M').mean()
    PLANT_NDEMAND=elm_output['PLANT_NDEMAND'].resample(time=str(smoothing)+'M').mean()
    NDEP_TO_SMINN=elm_output['NDEP_TO_SMINN'].resample(time=str(smoothing)+'M').mean()
    NFIX_TO_SMINN=elm_output['NFIX_TO_SMINN'].resample(time=str(smoothing)+'M').mean()
else:
    NET_NMIN=elm_output['NET_NMIN']
    GROSS_NMIN=elm_output['GROSS_NMIN']
    SMINN_TO_PLANT=elm_output['SMINN_TO_PLANT']
    PLANT_NDEMAND=elm_output['PLANT_NDEMAND']
    NDEP_TO_SMINN=elm_output['NDEP_TO_SMINN']
    NFIX_TO_SMINN=elm_output['NFIX_TO_SMINN']

(NET_NMIN*24*3600).plot(ax=ax,linestyle='-',color='blue',label='Net Nmin')
(GROSS_NMIN*24*3600).plot(ax=ax,linestyle='--',color='blue',label='Gross Nmin')
(SMINN_TO_PLANT*24*3600).plot(ax=ax,linestyle='-',color='green',label='Plant N uptake')
(PLANT_NDEMAND*24*3600).plot(ax=ax,linestyle='--',color='green',label='Plant N demand')

(NDEP_TO_SMINN*24*3600).plot(ax=ax,linestyle='-',color='C0',label='Atmospheric N dep')
(NFIX_TO_SMINN*24*3600).plot(ax=ax,linestyle='-',color='C1',label='N fixation')
ax.legend()
ax.set(title='N fluxes',xlabel='Year',ylabel='N flux (g N m$^{-2}$ day$^{-1}$)')

<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>